In [15]:
!pip install nltk

In [3]:
%%file bigram_analysis.py

from mrjob.job import MRJob
from mrjob.compat import jobconf_from_env
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import nltk

class MRBigramAnalysis(MRJob):
    
    def mapper_init(self):
        # Initialize NLTK resources
        nltk.download('punkt')
        nltk.download('stopwords')

    def mapper(self, _, line):
        # Tokenize the sentence
        words = word_tokenize(line.lower())

        # Remove stopwords and punctuation
        stop_words = set(stopwords.words('english'))
        words = [word for word in words if word.isalnum() and word not in stop_words]

        # Create bigrams
        bigrams = [(words[i], words[i + 1]) for i in range(len(words) - 1)]

        # Yield each bigram with count
        for bigram in bigrams:
            yield bigram, 1

    def reducer(self, bigram, counts):
        # Sum up the counts for each bigram
        yield None, (bigram, sum(counts))

    def reducer_top_bigrams(self, _, bigram_counts):
        # Sort bigrams by count in descending order and yield the top 20
        top_bigrams = sorted(bigram_counts, key=lambda x: x[1], reverse=True)[:20]
        for bigram, count in top_bigrams:
            yield bigram, count

if __name__ == '__main__':
    MRBigramAnalysis.run()

Overwriting bigram_analysis.py


In [ ]:
!python3 bigram_analysis.py SW_EpisodeIV.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/bigram_analysis.root.20231206.200538.429215
Running step 1 of 1...
[nltk_data] Downloading package punkt to /root/nltk_data...
